### Import Packages

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras
from tqdm import tqdm

2024-04-21 20:00:37.879113: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Load Data

In [2]:
# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)
n_residual_blocks = 5
# The data, split between train and test sets
mnist = keras.datasets.mnist
(x, _), (y, _) = mnist.load_data()
# Concatenate all the images together
data = np.concatenate((x, y), axis=0)
# Round all pixel values less than 33% of the max 256 value to 0
# anything above this value gets rounded up to 1 so that all values are either
# 0 or 1
data = np.where(data < (0.33 * 256), 0, 1)
data = data.astype(np.float32)

### PixelConvLayer

In [3]:
# The first layer is the PixelCNN layer. This layer simply builds on the 2D convolutional layer, but includes masking.
class PixelConvLayer(layers.Layer):
    def __init__(self, mask_type, **kwargs):
        super().__init__()
        self.mask_type = mask_type
        self.conv = layers.Conv2D(**kwargs)

    def build(self, input_shape):
        # Build the conv2d layer to initialize kernel variables
        self.conv.build(input_shape)
        # Use the initialized kernel to create the mask
        kernel_shape = tf.shape(self.conv.kernel)
        self.mask = np.zeros(shape=kernel_shape)
        self.mask[: kernel_shape[0] // 2, ...] = 1.0
        self.mask[kernel_shape[0] // 2, : kernel_shape[1] // 2, ...] = 1.0
        if self.mask_type == "B":
            self.mask[kernel_shape[0] // 2, kernel_shape[1] // 2, ...] = 1.0

    def call(self, inputs):
        self.conv.kernel.assign(self.conv.kernel * self.mask)
        return self.conv(inputs)


# Next, we build our residual block layer.
# This is just a normal residual block, but based on the PixelConvLayer.
class ResidualBlock(keras.layers.Layer):
    def __init__(self, filters, **kwargs):
        super().__init__(**kwargs)
        self.conv1 = keras.layers.Conv2D(
            filters=filters, kernel_size=1, activation="relu"
        )
        self.pixel_conv = PixelConvLayer(
            mask_type="B",
            filters=filters // 2,
            kernel_size=3,
            activation="relu",
            padding="same",
        )
        self.conv2 = keras.layers.Conv2D(
            filters=filters, kernel_size=1, activation="relu"
        )

    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.pixel_conv(x)
        x = self.conv2(x)
        return keras.layers.add([inputs, x])

### PixelCNN architecture

In [ ]:
inputs = keras.Input(shape=input_shape, batch_size=128)
x = PixelConvLayer(
    mask_type="A", filters=128, kernel_size=7, activation="relu", padding="same"
)(inputs)

for _ in range(n_residual_blocks):
    x = ResidualBlock(filters=128)(x)

for _ in range(2):
    x = PixelConvLayer(
        mask_type="B",
        filters=128,
        kernel_size=1,
        strides=1,
        activation="relu",
        padding="valid",
    )(x)

out = keras.layers.Conv2D(
    filters=1, kernel_size=1, strides=1, activation="sigmoid", padding="valid"
)(x)

pixel_cnn = keras.Model(inputs, out)
adam = keras.optimizers.Adam(learning_rate=0.0005)
pixel_cnn.compile(optimizer=adam, loss="binary_crossentropy")

pixel_cnn.summary()
pixel_cnn.fit(
    x=data, y=data, batch_size=128, epochs=50, validation_split=0.1, verbose=2
)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(128, 28, 28, 1)]        0         
                                                                 
 pixel_conv_layer (PixelCon  (128, 28, 28, 128)        6400      
 vLayer)                                                         
                                                                 
 residual_block (ResidualBl  (128, 28, 28, 128)        98624     
 ock)                                                            
                                                                 
 residual_block_1 (Residual  (128, 28, 28, 128)        98624     
 Block)                                                          
                                                                 
 residual_block_2 (Residual  (128, 28, 28, 128)        98624     
 Block)                                                      

### Display real images from MNIST

In [ ]:
real_images = np.concatenate((x, y), axis=0)
real_images = real_images.astype(np.float32)

# Select 10 random images from the dataset
indices = np.random.choice(range(len(real_images)), 10, replace=False)
sample_real_images = real_images[indices]

# Display these images in a 2x5 grid
fig, axes = plt.subplots(2, 5, figsize=(10, 4))
for i, ax in enumerate(axes.flatten()):
    ax.imshow(sample_real_images[i], cmap='gray')
    ax.axis('off')
plt.tight_layout()
plt.suptitle('Real Images from MNIST')
plt.show()

### Display generated images from MNIST

In [ ]:
generated_images = pixel_cnn.predict(np.random.rand(10, 28, 28, 1))  # Example generation

# Display these generated images in a 2x5 grid
fig, axes = plt.subplots(2, 5, figsize=(10, 4))
for i, ax in enumerate(axes.flatten()):
    ax.imshow(generated_images[i, :, :, 0], cmap='gray')
    ax.axis('off')
plt.tight_layout()
plt.suptitle('Generated Images from PixelCNN')
plt.show()

### Performance matrix

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np
import pandas as pd

# Assuming `real_images` is your batch of real images and `generated_images` is your batch of generated images
# Ensure both arrays are of the same shape

# Flatten the images for simplicity in calculation
real_images_flat = real_images.reshape(real_images.shape[0], -1)
generated_images_flat = generated_images.reshape(generated_images.shape[0], -1)

# Calculate Mean Squared Error (MSE) and then take the square root for RMSE
mse = mean_squared_error(real_images_flat, generated_images_flat)
rmse = np.sqrt(mse)

# Calculate Mean Absolute Error (MAE)
mae = mean_absolute_error(real_images_flat, generated_images_flat)

# Assuming binary cross-entropy loss, calculate it manually or retrieve it if you have it logged during training
# Here's a simplified manual calculation for binary cross-entropy loss
epsilon = 1e-7  # to prevent log(0)
bce_loss = -np.mean(
    real_images_flat * np.log(generated_images_flat + epsilon) + 
    (1 - real_images_flat) * np.log(1 - generated_images_flat + epsilon)
)

In [ ]:
metrics_dict = {
    'Metric': ['BCE Loss', 'RMSE', 'MAE'],
    'Value': [bce_loss, rmse, mae]
}

# Convert dictionary to DataFrame
metrics_df = pd.DataFrame(metrics_dict)

print(metrics_df)